## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-02-23-07-12-06 +0000,bbc,What the killing of drug lord 'El Mencho' mean...,https://www.bbc.com/news/videos/c0mg8d11pg3o?a...
1,2026-02-23-07-05-54 +0000,nyt,Duterte’s Fans Rally in The Hague as the I.C.C...,https://www.nytimes.com/2026/02/23/world/asia/...
2,2026-02-23-06-59-22 +0000,latimes,Magnitude 3.5 earthquake recorded off Catalina,https://www.latimes.com/california/story/2026-...
3,2026-02-23-06-53-36 +0000,bbc,Powerful winter storm slams US north-east as N...,https://www.bbc.com/news/articles/c75ere6z4weo...
4,2026-02-23-06-50-00 +0000,wsj,"Gold’s Bullish Structure Strengthens, Technica...",https://www.wsj.com/finance/commodities-future...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2506/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
186,trump,33
232,new,12
66,iran,12
74,hockey,11
73,olympic,9


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
25,2026-02-23-03-16-14 +0000,nyt,Armed Man Shot Dead at Mar-a-Lago by Trump’s S...,https://www.nytimes.com/2026/02/22/us/north-ca...,90
151,2026-02-22-14-09-27 +0000,cbc,"Armed man shot, killed after entering perimete...",https://www.cbc.ca/news/world/trump-mar-a-lago...,88
48,2026-02-23-02-00-00 +0000,wsj,With the bulk of President Trump’s tariffs str...,https://www.wsj.com/economy/trade/tariffs-are-...,85
105,2026-02-22-20-31-32 +0000,nypost,Trump congratulates USA men’s hockey team in p...,https://nypost.com/2026/02/22/sports/trump-con...,80
88,2026-02-22-22-08-03 +0000,bbc,Armed man killed after entering secure perimet...,https://www.bbc.com/news/articles/cp32l9w9pgyo...,70


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
25,90,2026-02-23-03-16-14 +0000,nyt,Armed Man Shot Dead at Mar-a-Lago by Trump’s S...,https://www.nytimes.com/2026/02/22/us/north-ca...
85,62,2026-02-22-22-13-39 +0000,nypost,Panicked passengers run for lives at Guadalaja...,https://nypost.com/2026/02/22/world-news/panic...
77,50,2026-02-22-22-52-24 +0000,nypost,"White House trolls Canada, Justin Trudeau with...",https://nypost.com/2026/02/22/sports/white-hou...
48,40,2026-02-23-02-00-00 +0000,wsj,With the bulk of President Trump’s tariffs str...,https://www.wsj.com/economy/trade/tariffs-are-...
154,31,2026-02-22-14-00-40 +0000,bbc,Iran students stage first large anti-governmen...,https://www.bbc.com/news/articles/c5yj2kzkrj0o...
14,30,2026-02-23-05-30-58 +0000,bbc,Violence erupts in Mexico after drug lord El M...,https://www.bbc.com/news/articles/c795qgejzpxo...
6,30,2026-02-23-06-42-06 +0000,nypost,Grieving North Carolina dad attacks teen son’s...,https://nypost.com/2026/02/23/us-news/north-ca...
55,24,2026-02-23-01-19-49 +0000,nypost,Chaos at LAX after TSA PreCheck and Global Ent...,https://nypost.com/2026/02/22/us-news/lax-mayh...
41,24,2026-02-23-02-24-27 +0000,nypost,Californians hit the bars before breakfast to ...,https://nypost.com/2026/02/22/us-news/obriens-...
54,24,2026-02-23-01-21-34 +0000,nypost,Top doctor reveals exactly why avalanche morta...,https://nypost.com/2026/02/22/us-news/top-doct...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
